In [ ]:
# Imports

import car
from auto.capabilities import acquire, release, list_caps
from car.motors import set_throttle, set_steering
from auto.camera import global_camera, close_global_camera
from auto import console
from typing import Tuple
import time

import numpy as np
import cv2
import os

list_caps()

('AHRS',
 'Accelerometer',
 'Buzzer',
 'Calibrator',
 'Camera',
 'CarMotors',
 'Credentials',
 'Encoders',
 'Gyroscope',
 'Gyroscope_accum',
 'LEDs',
 'LoopFrequency',
 'PID_steering',
 'PWMs',
 'Photoresistor',
 'Power',
 'PushButtons',
 'VersionInfo')

In [ ]:
# Computer Vision constants
cv2.setNumThreads(0)

# BLUE = np.array([112,121, 189])
left_color = np.array([-1, -1, -1])
right_color = np.array([-1, -1, -1])
turn_color = np.array([-1, -1, -1])
straight_color = np.array([-1, -1, -1])
car_color = np.array([-1, -1, -1])

DIFF = 30

def determine_bounds(color, diff):
    lower = np.clip(color - diff, 0, 255)
    upper = np.clip(color + diff, 0, 255)
    return lower, upper

left_color_lower, left_color_upper = determine_bounds(left_color, DIFF)
right_color_lower, right_color_upper = determine_bounds(right_color, DIFF)
turn_color_lower, turn_color_upper = determine_bounds(turn_color, DIFF)
straight_color_lower, straight_color_upper = determine_bounds(straight_color, DIFF)
car_color_lower, car_color_upper = determine_bounds(car_color, DIFF)

CROP_HEIGHT = 70
CROP_LEFT = 60
CROP_RIGHT = 60

In [ ]:
# Functions

def track_bounds_crop(img):
    """
    Camera captures 240 pixels of height and 320 of width
    Shape is (240, 320, 3)
    """
    return img[-CROP_HEIGHT:, CROP_LEFT:-CROP_RIGHT]


def track_zones_crop(img):
    """
    Cut out the top half
    """
    return img[-120:, :]


def scale(img, scale_percent):
    # Calculate the new dimensions
    width = int(img.shape[1] * scale_percent)
    height = int(img.shape[0] * scale_percent)
    new_size = (width, height)

    # Resize the image:
    img = cv2.resize(img, new_size, None, None, None, cv2.INTER_AREA)
    return img


def percent_color(img, lower_bound, upper_bound, scale_percent=1):
    rgb_image = img
    rgb_mask = cv2.inRange(rgb_image, lower_bound, upper_bound)
    ratio = cv2.countNonZero(rgb_mask)/(img.size/3)
    color_percent = (ratio * 100) / scale_percent 
    return color_percent


def throttle_to_feet_per_second(throttle):
    return 0.33514 * throttle - 3.39442


def feet_per_second_to_throttle(feet_per_second):
    return (feet_per_second + 3.39442) / 0.33541


def identify_opposing_cars(frame, threshold) -> Tuple[bool, bool, bool, bool, bool]:
    """
    Returns a tuple of 5 booleans indicating whether there is an opposing car in each of 5 vertical zones
    Meaning the very leftmost and rightmost areas (not a threat), middle left + right (should move over), and middle (get the heck out of dodge)
    """

    # Identify the percentage of pixels in each zone that are the color of the car
    width = frame.shape[1]
    height = frame.shape[0]

    left_zone = frame[:, :width//5]
    right_zone = frame[:, -width//5:]
    middle_left_zone = frame[:, width//5:2*width//5]
    middle_right_zone = frame[:, 3*width//5:4*width//5]
    middle_zone = frame[:, 2*width//5:3*width//5]

    left_zone_percent = percent_color(left_zone, car_color_lower, car_color_upper)
    right_zone_percent = percent_color(right_zone, car_color_lower, car_color_upper)
    middle_left_zone_percent = percent_color(middle_left_zone, car_color_lower, car_color_upper)
    middle_right_zone_percent = percent_color(middle_right_zone, car_color_lower, car_color_upper)
    middle_zone_percent = percent_color(middle_zone, car_color_lower, car_color_upper)

    return (left_zone_percent > threshold, right_zone_percent > threshold, middle_left_zone_percent > threshold, middle_right_zone_percent > threshold, middle_zone_percent > threshold)


In [ ]:
# Racing variables

# This is the only information needed to turn the corner
turn_radii = []
turns = []

NUM_TURNS = 10
NUM_LAPS = 5
MAX_THROTTLE = 22
MIN_THROTTLE = 18

STATIC_FRICTION = 0.7
GRAVITY = 9.81
WHEEL_BASE = 6.5 / 12 # 6.5 inches

In [ ]:
# Custom logic
THROTTLE_STEP = 5
ANGLE_THRESHOLD = 15.0 # If greater than this on a straight and approaching a boundary reduce the throttle

TURN_COLOR_THRESHOLD = float('inf') # As a percent
STRAIGHT_COLOR_THRESHOLD = float('inf') # As a percent

CAR_THRESHOLD = float('inf') # As a percent

STRAIGHT_STEER_BIAS = -6.0

state = "STRAIGHT" # current driving mode
turn_index = 0

def loop(prev_throttle, gyroscope, angle_accum, frame):
    """
    Main control loop for Robocar F1:
    - Blue marks track boundaries
    - StartTurnColor marks the START/END of a turn
    """
    global state, turn_index, lap
    
    steer_bias = 0

    throttle = prev_throttle
    steer = 0
    steer_direction = 1

    car_positions = identify_opposing_cars(frame, CAR_THRESHOLD)

    # Check for instructions to change state
    bottom_portion_img = track_zones_crop(frame)
    if percent_color(bottom_portion_img, turn_color_lower, turn_color_upper) > TURN_COLOR_THRESHOLD:
        # Start turning + update angle_accum
        state = "TURN"
        angle_accum += turns[turn_index]
        turn_index += 1 # While we want to increment the turn index, we don't want to wrap around to 0 completing the lap until we hit the straight
        
    elif percent_color(bottom_portion_img, straight_color_lower, straight_color_upper) > STRAIGHT_COLOR_THRESHOLD:
        # Exit turn and update turn + lap variables
        state = "STRAIGHT"
        if turn_index == NUM_TURNS:
            turn_index = 0
            lap += 1

    track_bounds_img = track_bounds_crop(frame)
    if percent_color(track_bounds_img, left_color_lower, left_color_upper):
        bound = "LEFT"
    elif percent_color(track_bounds_img, right_color_lower, right_color_upper):
        bound = "RIGHT"
    else:
        bound = "NONE"

    if state == "STRAIGHT":
        steer_bias = STRAIGHT_STEER_BIAS
        
        z = gyroscope.read()[2]
        if z - angle_accum > 0:
            steer_direction = -1

        steer = np.abs(z) / 2
        
        if bound != "NONE" and steer > ANGLE_THRESHOLD:
            throttle -= THROTTLE_STEP
        else:
            throttle += THROTTLE_STEP

        throttle = min(max(MIN_THROTTLE, throttle), MAX_THROTTLE) # Clip it on either side

        # For now don't overtake for testing purposes, so just reduce throttle if any cars can be seen
        if any(car_positions):
            throttle -= 2 * THROTTLE_STEP # Has to be 2x here because above throttle keeps on being pushed

    # Turns
    elif state == "TURN":
        """
        Due to error and good racing doesn't follow the perfect line, this can be deviated from, but anything widely
        off of this angle will be very very inefficient and maybe even dangerous
        """
        
        throttle_bias = 0
        turn_radius = turn_radii[turn_index]
        steer_direction = -1 if turns[turn_index] > 0 else 1

        if bound == "LEFT":
            # If we're on the left side of the track, we need to steer more to the right
            # Increase radius if we're turning left, decrease if we're turning right
            turn_radius = turn_radius * 1.1 if turns[turn_index] > 0 else turn_radius * 0.9
            throttle_bias = -THROTTLE_STEP
        elif bound == "RIGHT":
            # If we're on the right side of the track, we need to steer more to the left
            # Increase radius if we're turning right, decrease if we're turning left
            turn_radius = turn_radius * 0.9 if turns[turn_index] > 0 else turn_radius * 1.1
            throttle_bias = THROTTLE_STEP

        # Convert to deg
        steer = np.arctan(WHEEL_BASE / turn_radius) * 180 / np.pi
        steer = 0.597753 * steer - 2.22879
        throttle = feet_per_second_to_throttle(np.sqrt(turn_radius * STATIC_FRICTION * GRAVITY))
        
        # For now don't overtake for testing purposes, so just reduce throttle if any cars can be seen
        if any(car_positions):
            throttle_bias += THROTTLE_STEP

        throttle += throttle_bias

    throttle = np.clip(throttle, MIN_THROTTLE, MAX_THROTTLE)

    return (steer * steer_direction) + steer_bias, throttle, angle_accum


# on_straight = False
# StartTurnColor_WAIT = 1.0 # Can't begin detecting for a corner start/end until we wait this long
# last_StartTurnColor_detection = 0.0

# turn_index = 0 # Index into turns array
# lap = 0

# STRAIGHT_STEER_BIAS = -6.0 # The measured offset that the car is from straight (this is a motor problem, the gyroscope understands that it's 6 degrees off)

# BLUE_THRESHOLD = 1.0 # As a percent
# StartTurnColor_THRESHOLD = 100.0 # As a percent
# ANGLE_THRESHOLD = 15.0 # If greater than this on a straight and approaching a boundary reduce the throttle

# def start():
#     return 0, MAX_THROTTLE


# def loop(prev_throttle, on_straight, gyroscope, angle_accum, frame, delta):
#     global last_StartTurnColor_detection
#     global turn_index
#     steer = 0
#     steer_direction = 1
    
#     throttle = prev_throttle
    
#     if time.time() - last_StartTurnColor_detection >= StartTurnColor_WAIT:
#         StartTurnColor_img = bottom_portion(frame)
#         StartTurnColor_percent = percent_color(StartTurnColor_img, StartTurnColor_LOWER_BOUND, StartTurnColor_UPPER_BOUND)
#         if StartTurnColor_percent > StartTurnColor_THRESHOLD:
#             on_straight = not on_straight
#             last_StartTurnColor_detection = time.time()

#             if on_straight:
#                 angle_accum += turns[turn_index] # We have to do it like this in case the car isn't straight coming out of the corner
#                 turn_index += 1
#                 if turn_index >= num_turns:
#                     turn_index = 0
#                     lap += 1

#                 throttle = MAX_THROTTLE
        

    # For Straight
#     if state == "STRAIGHT":
#         steer_bias = -6.0
        
#         # Smooth acceleration
#         throttle = min(throttle + THROTTLE_STEP, MAX_THROTTLE)
        
#         # Center between blue boundaries
#         blue_mask = cv2.inRange(blue_img, S_COLOR_LOWER_BOUND, S_COLOR_UPPER_BOUND)
#         M = cv2.moments(blue_mask)
#         if M["m00"] != 0:
#             cx = int(M["m10"] / M["m00"])
#             center_offset = cx - (blue_img.shape[1] // 2)
#             steer = center_offset / 20.0

#         # Slow slightly if drifting too far
#         if abs(z_angle - angle_accum) > ANGLE_THRESHOLD:
#             throttle = max(throttle - 5, MIN_THROTTLE)
                
#     else:
#         """
#         Formula for theoretical turn angle is (v * dt / r)
#         Due to error and good racing doesn't follow the perfect line, this can be deviated from, but anything widely
#         off of this angle will be very very inefficient and maybe even dangerous
#         """
#         turn_radius = turn_radii[turn_index]
#         steer_direction = -1 if turns[turn_index] > 0 else 1
    
#         throttle = 60

#         # Would be nice if we could two pieces of color tape on the corner, one blue and one another color (not StartTurnColor)

#         steer = throttle_to_feet_per_second(throttle) * delta / turn_radius
#         # Steer is in radians, convert to deg
#         steer *= 180 / np.pi
#         return (steer * steer_direction) + 16, throttle, on_straight, angle_accum
     

In [ ]:
console.clear()

gyroscope = acquire('Gyroscope_accum')
gyroscope.read()

camera = global_camera(False)
stream = camera.stream() # Returns a generator
next(stream)

throttle = MIN_THROTTLE

angle_accum = 0
lap = 0
while lap <= NUM_LAPS:
    if stream:
        steer, throttle, angle_accum = loop(throttle, gyroscope, angle_accum, next(stream))
        if lap == NUM_LAPS:
            set_throttle(int(throttle * 0.4))  # convert to native int
        else:
            set_throttle(int(throttle))  # convert to native int
        set_steering(steer)

release(gyroscope)
close_global_camera(False)

KeyboardInterrupt: 

In [18]:
while True:
    frame = car.capture()
    bottom = bottom_portion(frame)
    console.print(percent_color(bottom, StartTurnColor_LOWER_BOUND, StartTurnColor_UPPER_BOUND))


Captured 1 frame.


NameError: name 'StartTurnColor_LOWER_BOUND' is not defined

In [13]:
while True:
    set_steering(-5)

KeyboardInterrupt: 

In [47]:
console.clear()

IndentationError: expected an indented block (<ipython-input-1-e546faa82dc9>, line 75)